In [1]:
%cd /home/jovyan/work

from bokeh.io import output_notebook

output_notebook()


/home/jovyan/work


Loading BokehJS ...

In [2]:
from bokeh.io import show
from bokeh.plotting import figure
from bokeh.util.hex import axial_to_cartesian
from bokeh.models import HoverTool

from chordifier.KeyboardRenderer import KeyboardRenderer
from chordifier.algorithm.Sequencer import Sequencer

import numpy as np

np.set_printoptions(precision=4, suppress=True)


def present(keyboard):
    renderer = KeyboardRenderer(keyboard)

    f = make_figure()
    draw_hexagons(f, renderer.qs, renderer.rs, renderer.colors, renderer.positions)
    return f

def make_figure():
   
    f = figure(tools="pan,wheel_zoom,reset", height=600,
               sizing_mode='stretch_width', match_aspect=True)
    f.grid.visible = False
    quad = f.quad(top=[15], bottom=[-16], left=[-25],
       right=[9], color="#000000", alpha=0)
    
    hover = HoverTool()
    hover.tooltips = [
        ("x", "$x"),
        ("y", "$y"),
    ]
    hover.renderers = [quad]
    hover.point_policy = "follow_mouse"
    hover.attachment = "right"
    f.add_tools(hover)

    return f

def draw_hexagons(f, q, r, colors, labels):
    f.hex_tile(q, r, fill_color=colors, alpha=0.5, line_color="white",
               orientation="flattop")

    x, y = axial_to_cartesian(q, r, 1, "flattop")

    f.text(x, y, text=labels,
           text_baseline="middle", text_align="center")


In [3]:
from chordifier.algorithm.Pipeline import Pipeline
from chordifier.experiments.KeyboardFactory import make
from chordifier.utils import vector

PARAMETERS = {
    # general
    'keyboard': make([3, 3, 3, 3, 2] + [0] * 5),
    'characters': 15,

    # sequencer
    'filename': "dataset/ngram.json",
    'length': 3,
    'samples': 500,

    # dynamics
    'x_y_ratio': 1.5,
    'stiffness': vector(1.8, 1.3, 1.2, 1.1, 2, 2, 1.1, 1.2, 1.3, 1.8),

    # pruner
    'priority': vector(18000, 13000, 12000, 11000, 14500,
                       14516, 11008, 12004, 13002, 18001),
    'finger_priorities': 1,
    'average_offsets': 1,
    'deviation_of_offsets': 1,

    # evaluator
    'distances_travelled': 1,
    'chord_difficulties': 1,

    # optimizer
    'best': True,
    'generations': 5,
    'population_size': 100,
    'mate_probability': 0.5,
    'mutate_probability': 0.20,
    'mutate_independent_probability': 0.10,
    'select_tournament_size': 10,
}

pipeline = Pipeline(PARAMETERS)

pipeline.prepare()
result = pipeline.optimize()


gen	nevals	average    	standard   	minimum    	maximum    
0  	100   	3.03089e+08	1.64935e+07	2.65489e+08	3.46376e+08
1  	64    	2.86048e+08	1.1161e+07 	2.65489e+08	3.18025e+08
2  	53    	2.80972e+08	1.46225e+07	2.56823e+08	3.30403e+08
3  	67    	2.7835e+08 	1.87538e+07	2.56823e+08	3.32007e+08
4  	64    	2.72596e+08	1.92085e+07	2.50966e+08	3.27536e+08
5  	57    	2.61741e+08	1.42069e+07	2.50034e+08	3.26956e+08


In [4]:
s = Sequencer("dataset/ngram.json", 4, 3, 9)

print("=== Sequencer test ===")
print("--- sequences ---")
print(s.sequences)
print("--- occurrences ---")
print(s.occurrences)
print("--- uniques ---")
print(s.uniques)
print("--- indices ---")
print(s.indices)


=== Sequencer test ===
--- sequences ---
[['e' 't' '_']
 ['e' 't' ' ']
 ['t' 'e' '_']
 ['e' ' ' 't']
 ['t' 'e' ' ']
 ['t' ' ' 't']
 ['_' 't' ' ']
 [' ' '_' '_']
 ['e' '_' 't']]
--- occurrences ---
[766134 519918 484094 484074 435720 308600 248276 246470 222423]
--- uniques ---
[' ' '_' 'e' 't']
--- indices ---
[[2 3 1]
 [2 3 0]
 [3 2 1]
 [2 0 3]
 [3 2 0]
 [3 0 3]
 [1 3 0]
 [0 1 1]
 [2 1 3]]


In [5]:
print("=== Sequencer ===")
print("--- sequences ---")
print(pipeline.sequencer.sequences[:10])
print("--- occurrences ---")
print(pipeline.sequencer.occurrences[:10])
print("--- uniques ---")
print(pipeline.sequencer.uniques)
print("--- indices ---")
print(pipeline.sequencer.indices[:10])

=== Sequencer ===
--- sequences ---
[['i' 'n' 'e']
 ['n' 'e' ' ']
 ['s' 't' 'r']
 ['s' 't' 'a']
 ['i' 'n' 't']
 ['c' 't' ' ']
 ['t' 'a' 't']
 ['\n' 's' 't']
 ['c' 'o' 'n']
 ['_' 'r' 'e']]
--- occurrences ---
[2576070 2408500 2159460 1958257 1852123 1825148 1592096 1517714 1488500
 1438328]
--- uniques ---
['\n' ' ' '_' 'a' 'c' 'd' 'e' 'i' 'l' 'n' 'o' 'p' 'r' 's' 't']
--- indices ---
[[ 7  9  6]
 [ 9  6  1]
 [13 14 12]
 [13 14  3]
 [ 7  9 14]
 [ 4 14  1]
 [14  3 14]
 [ 0 13 14]
 [ 4 10  9]
 [ 2 12  6]]


In [6]:
print("=== Keyboard ===")
f = present(pipeline.keyboard)
show(f)


=== Keyboard ===


In [7]:
print("=== Preprocessor ===")
print("--- zones ---")
for zone in pipeline.preprocessor.zones:
    print(zone)
print("--- chords ---")
print(pipeline.preprocessor.chords[222:999:666])
print("--- positions ---")
print(pipeline.preprocessor.positions[222:999:666])
print("--- origins ---")
print(pipeline.preprocessor.origins)


=== Preprocessor ===
--- zones ---
[[-8  4]
 [-8  3]
 [-9  4]
 [-9  5]
 [-8  5]]
[[-7  3]
 [-7  2]
 [-7  4]]
[[-6  2]
 [-6  1]
 [-6  3]]
[[-5  2]
 [-5  1]
 [-4  1]
 [-4  2]
 [-5  3]]
[[-3  3]
 [-3  2]
 [-2  2]
 [-2  3]
 [-3  4]
 [-4  4]
 [-4  3]]
[]
[]
[]
[]
[]
--- chords ---
[[0 1 0 3 7 0 0 0 0 0]
 [1 0 2 3 1 0 0 0 0 0]]
--- positions ---
[[[     nan      nan]
  [-10.5      0.866 ]
  [     nan      nan]
  [ -6.       1.7321]
  [ -6.      -1.7321]
  [     nan      nan]
  [     nan      nan]
  [     nan      nan]
  [     nan      nan]
  [     nan      nan]]

 [[-12.      -0.    ]
  [     nan      nan]
  [ -9.       3.4641]
  [ -6.       1.7321]
  [ -4.5     -2.5981]
  [     nan      nan]
  [     nan      nan]
  [     nan      nan]
  [     nan      nan]
  [     nan      nan]]]
--- origins ---
[[-12.      -0.    ]
 [-10.5      0.866 ]
 [ -9.       1.7321]
 [ -7.5      0.866 ]
 [ -4.5     -2.5981]
 [     nan      nan]
 [     nan      nan]
 [     nan      nan]
 [     nan      nan]
 [     na

In [8]:
np.set_printoptions(suppress=True)
print("=== Pruner intact ===")
print("--- chords ---")
print(pipeline.pruner_intact.chords[222:999:666])
print("--- positions ---")
print(pipeline.pruner_intact.positions[222:999:666])
print("--- origins ---")
print(pipeline.pruner_intact.origins)
print("--- metrics ---")
print(pipeline.pruner_intact.metrics[222:999:666])
print("--- scores ---")
print(pipeline.pruner_intact.scores[222:999:666])
print("--- totals ---")
print(pipeline.pruner_intact.totals[222:999:666])


=== Pruner intact ===
--- chords ---
[[0 1 0 3 7 0 0 0 0 0]
 [1 0 2 3 1 0 0 0 0 0]]
--- positions ---
[[[     nan      nan]
  [-15.75     0.866 ]
  [     nan      nan]
  [ -9.       1.7321]
  [ -9.      -1.7321]
  [     nan      nan]
  [     nan      nan]
  [     nan      nan]
  [     nan      nan]
  [     nan      nan]]

 [[-18.      -0.    ]
  [     nan      nan]
  [-13.5      3.4641]
  [ -9.       1.7321]
  [ -6.75    -2.5981]
  [     nan      nan]
  [     nan      nan]
  [     nan      nan]
  [     nan      nan]
  [     nan      nan]]]
--- origins ---
[[-18.      -0.    ]
 [-15.75     0.866 ]
 [-13.5      1.7321]
 [-11.25     0.866 ]
 [ -6.75    -2.5981]
 [     nan      nan]
 [     nan      nan]
 [     nan      nan]
 [     nan      nan]
 [     nan      nan]]
--- metrics ---
[[38500.         2.4913     1.8819]
 [55500.         1.1826     1.2103]]
--- scores ---
[[0.562  0.5167 0.7806]
 [0.8102 0.2453 0.502 ]]
--- totals ---
[1.8593 1.5575]
